Se tiene que realizar el desarrollo  dos modelos de recomendación. Estos modelos generan listas de 5 juegos al ingresar el nombre de un juego o el ID de un usuario.

En el primer caso, el modelo utiliza una relación ítem-ítem, sugiriendo juegos similares basándose en la similitud con el juego proporcionado.

En el segundo caso, el modelo emplea un filtro usuario-juego, identificando usuarios similares y recomendando juegos que hayan sido apreciados por esos usuarios.

Para lo anterior se hará uso de algoritmos basados en la memoria, filtrando mediante el análisis de toda la base de datos. El objetivo es encontrar usuarios similares al usuario activo y utilizar sus preferencias para prever las valoraciones del usuario activo.

In [1]:
# Bibliotecas

import pandas as pd
import numpy as np

import scipy as sp
from sklearn.metrics.pairwise import cosine_similarity
import operator

import pyarrow as pa
import pyarrow.parquet as pq

In [2]:
# Para el modelo abriré el df que se realizó para el modelo

ruta_parquet = r"C:\Users\Usuario\Henry\PI1_ML\Modelo_Aprendizaje_Automatico\Datos_Modelo\df_modelo.parquet"

# Leer el archivo CSV y convertirlo en un DataFrame
df_modelo = pd.read_parquet(ruta_parquet)



In [3]:
df_modelo.head()

,user_id,item_name,rating
0,76561197970982479,Killing Floor,3
1,js41637,Barbie™ Dreamhouse Party™,3
2,evcentric,Risk of Rain,5
3,doctr,The Wolf Among Us,5
4,maplemage,Dark Souls: Prepare to Die Edition,3


In [5]:
piv = df_modelo.pivot_table(index=['user_id'], columns=['item_name'], values='rating')
piv.sample(20)

item_name,! That Bastard Is Trying To Steal Our Gold !,0RBITALIS,"10,000,000",100% Orange Juice,1001 Spikes,12 Labours of Hercules,12 Labours of Hercules II: The Cretan Bull,12 is Better Than 6,123 Slaughter Me Street,140,...,klocki,liteCam Game: 100 FPS Game Capture,oO,planetarian ~the reverie of a little planet~,resident evil 4 / biohazard 4,sZone-Online,the static speaks my name,theBlu,theHunter,theHunter: Primal
user_id,,,,,,,,,,,,,,,,,,,,,
OGDelicate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
RonaldMcDonaldTouchesKids,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
76561198056545854,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Hummingsenpai,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GN1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
76561197967708845,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
76561198069465728,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
76561198057723496,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Kyuok,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Se normalizan los valores del dataframe piv restar la media de las calificaciones de un usuario y luego dividir
por la diferencia entre el valor máximo y mínimo de las calificaciones. Esto ajusta las calificaciones de un usuario 
y se centradan   en cero y escaladas en función de su variabilidad. 
A los usuarios que solo han dado una calificación o han calificado todos los juegos de la misma manera serán eliminados
durante este proceso de normalización. Esto se debe a que estos usuarios no aportan información útil para el modelo de recomendación si todas sus calificaciones son iguales o si solo tienen una calificación.

In [6]:
# Normalización del dataframe 'piv'
piv_norm = piv.apply(lambda x: (x-np.mean(x))/(np.max(x)-np.min(x)), axis=1)
# Se borran las columnas que contienen solo cero o no tienen rating, se rellenan los vacios con 0 y se hace la transpuesta
piv_norm.fillna(0, inplace=True)
piv_norm = piv_norm.T
piv_norm = piv_norm.loc[:, (piv_norm != 0).any(axis=0)]
piv_norm

user_id,--000--,-Beave-,-GM-Dragon,-I_AM_EPIC-,-SEVEN-,-Thyme-,-kainey9777,00000000000000000001227,00690069006900,03092002,...,zomgCoBfAce,zoom-the-flash,zoozles,zourock,zrustz16,zsharoarkbr,zuzuga2003,zvanik,zwanzigdrei,zzoptimuszz
item_name,,,,,,,,,,,,,,,,,,,,,
! That Bastard Is Trying To Steal Our Gold !,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0RBITALIS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"10,000,000",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100% Orange Juice,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1001 Spikes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
sZone-Online,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
the static speaks my name,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
theBlu,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Necesito convierte a un formato de matriz dispersa (sparse matrix) este df normalizado para reducir la memoria utilizada  La matriz dispersa es un tipo de estructura de datos que almacena solo los valores distintos de cero junto con su ubicación en la matriz, en lugar de almacenar todos los valores de la matriz, incluso los ceros.

In [7]:
piv_sparse = sp.sparse.csr_matrix(piv_norm.values)
piv_sparse

<3294x8030 sparse matrix of type '<class 'numpy.float64'>'
	with 30699 stored elements in Compressed Sparse Row format>

In [ ]:
#A los datos de esta matriz normalizada se los convierte a un formato de matriz dispersa (sparse matrix) para reducir la memoria utilizada y mejorar la eficiencia en el manejo de grandes conjuntos de datos, especialmente cuando la mayoría de los valores en la matriz son ceros. La matriz dispersa es un tipo de estructura de datos que almacena solo los valores distintos de cero junto con su ubicación en la matriz, en lugar de almacenar todos los valores de la matriz, incluso los ceros.

Se deben desarrollar dos matrices de similitud utilizando la similitud del coseno para medir la similitud entre los juegos (item_similarity) y entre los usuarios (user_similarity).

La similitud del coseno es una medida comúnmente utilizada para evaluar la similitud entre dos vectores en un espacio multidimensional. En el contexto de sistemas de recomendación y análisis de datos, la similitud del coseno se utiliza para determinar cuán similares son dos conjuntos de datos o elementos, y se calcula utilizando el coseno del ángulo entre los vectores que representan esos datos o elementos.

In [8]:
item_similarity = cosine_similarity(piv_sparse)
user_similarity = cosine_similarity(piv_sparse.T)

In [9]:
#  se insertan las matrices anteriores en un Dataframe. para hacerlo mas entendible
item_sim_df = pd.DataFrame(item_similarity, index = piv_norm.index, columns = piv_norm.index)
#user similarity dataframe
user_sim_df = pd.DataFrame(user_similarity, index = piv_norm.columns, columns = piv_norm.columns)

FUNCION RECOMENDACION DE JUEGO

Con la relación entre los distintos juegos, se desarrolla una función que realice una recomendación de 5 juegos en función de un juego dado, teniendo en cuenta los valores mas altos de similitud del coseno. Se tirnr un nombre de juego como argumento, se ordena la columna correspondiente de ese juego en la matriz de similitud entre elementos (item_sim_df) de manera descendente, de modo que los juegos más similares aparezcan en la parte superior. Posteriormente selecciona los 5 juegos más similares (excluyendo el propio juego que se pasó como entrada), itera a través de estos juegos similares y, finalmente, imprime una lista de juegos similares al juego especificado.

In [10]:
def top_game(game):
    '''
    Muestra una lista de juegos similares a un juego dado.

    Args:
        game (str): El nombre del juego para el cual se desean encontrar juegos similares.

    Returns:
        None: Esta función imprime una lista de juegos 5 similares al dado.

    '''
    count = 1
    print('Similar games to {} include:\n'.format(game))
    for item in item_sim_df.sort_values(by = game, ascending = False).index[1:6]:
        print('No. {}: {}'.format(count, item))
        count +=1

In [11]:
# ejemplo
top_game('Metro 2033')

Similar games to Metro 2033 include:

No. 1: Gotham City Impostors
No. 2: Space Hulk
No. 3: S.T.A.L.K.E.R.: Shadow of Chernobyl
No. 4: You Must Build A Boat
No. 5: Killing Floor


In [ ]:
# Lo unico que queda es incluir esta función en el main